In [207]:
from juritagger import JuriMatcher
%load_ext autoreload
%autoreload 2
jm = JuriMatcher(["./dicos/merged.txt", "./dicos/cluster_merged_classif.json"], "fr_core_news_md")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [221]:
fiche_text = """Fermeture de l'entreprise pour congés: comment le salarié est-il indemnisé?
L'indemnisation du salarié dépend du nombre de jours de fermeture
de l'entreprise et du nombre de jours de congés acquis par le salarié.
Si le salarié possède suffisamment de jours de congés par rapport au
nombre de jours de fermeture de congés, il perçoit l'indemnité de congés payés dans les conditions habituelles. 
Si le salarié n'a pas acquis suffisamment de jours de congés payés pour être indemnisé
durant l'intégralité de la fermeture de l'entreprise, Pôle emploi peut verser, sous conditions,
une aide financière pour congés non payés. Cette aide pour congés non payés est versée au salarié qui percevait,
avant sa reprise d'emploi, l'allocation d'aide au retour à l'emploi (ARE)  ou l'allocation de solidarité spécifique
(ASS). Si c'était le cas, le salarié doit formuler une demande d'aide auprès de l'agence Pôle emploi
dont il dépendait en tant que demandeur d'emploi. Pôle emploi pole_emploi Pôle emploi 
Le montant de l'aide est calculé par Pôle emploi qui tient compte du nombre de jours de fermeture
de l'entreprise; net des droits à congés payés acquis par le salarié au titre de son nouvel emploi.
 À défaut d'avoir droit à l'aide pour congés non payés, le salarié peut demander à son employeur de bénéficier 
de congés payés par anticipation, mais l'employeur n'est pas obligé d'accepter (sauf accord collectif ou
usages contraires). Le salarié peut également demander à bénéficier d'un congé sans solde ou d'une indemnité d'activité partielle,
en cas d'intempérie à caractère exceptionnel notamment."""

matches, doc = jm.tag_doc(fiche_text, mode = "flat")

In [209]:
class MyDoc:
    def __init__(self, doc):
        self.doc = doc
        
    def __getitem__(self, item):
        return self.doc[item]
    
    def _split(self, s=None):
        return self.doc.text.split(s)
    
    def __add__(self, o):
        return self.doc.text + o
    
    def __radd__(self, o):
        return o + self.doc.text

In [210]:
doc2 = MyDoc(doc)

In [211]:
def check_overlap(i, entity_range):
    v = []
    entity = entity_range[i]
    for j, a in enumerate(entity_range):
        if j != i and entity.intersection(a) != set():
            v.append(j)
    return v

def keep_longer_match(matches):
    entity_range = []
    for label, start, end in matches:
        entity_range.append(set(range(start, end)))

    new_matches = []
    for i in range(len(entity_range)):
        overlaps = check_overlap(i, entity_range)
        longer = [len(entity_range[over]) > len(entity_range[i]) for over in overlaps]
        equal = [len(entity_range[over]) == len(entity_range[i]) for over in overlaps]
        if any(longer):
            continue
            
        if any(equal):
            if matches[i] not in new_matches:
                new_matches.append(matches[i])
        
        else:
            new_matches.append(matches[i])
    return new_matches

In [212]:
matches = keep_longer_match(matches)

In [213]:
jm.entity_types = ["JUR"]

from display_entities import COLOR_LIST
options = {
    "ents": list(jm.entity_types),
    "colors" : {entity:COLOR_LIST[i] for i, entity in enumerate(jm.entity_types)}
}

In [214]:
doc[matches[0][1]:matches[0][2]]

congés

In [215]:
jm.flat_matcher.get("JUR")

(<bound method JuriMatcher._add_event_ent of <juritagger.JuriMatcher object at 0x187a9ae10>>,
 [[(1, [(66, 1711323367175830939)]),
   (1, [(66, 5890906176591112566)]),
   (1, [(66, 464273237196873268)])],
  [(1, [(66, 1793048885839419356)]),
   (1, [(66, 14626626061804382878)]),
   (1, [(66, 15433310777704271958)])],
  [(1, [(66, 16317943348733762443)])],
  [(1, [(66, 11240525707520994885)]), (1, [(66, 17582643838483714874)])],
  [(1, [(66, 1793048885839419356)]), (1, [(66, 9100968497564308803)])],
  [(1, [(66, 6630227242837467831)]), (1, [(66, 6831987452066723016)])],
  [(1, [(66, 6630227242837467831)]),
   (1, [(66, 11144093025662894627)]),
   (1, [(66, 3267034769997045434)])],
  [(1, [(66, 11956852170057626367)]), (1, [(66, 10165509397764927418)])],
  [(1, [(66, 11372891394010144125)]),
   (1, [(66, 6199346620330166190)]),
   (1, [(66, 6076196367831591072)])],
  [(1, [(66, 18372220980445018914)])],
  [(1, [(66, 15304760848371220175)]), (1, [(66, 2899740202173365939)])],
  [(1, [(66,

In [216]:
from display_entities import COLOR_LIST
options = {
    "ents": list(jm.entity_types),
    "colors" : {entity:COLOR_LIST[i] for i, entity in enumerate(jm.entity_types)}
}

In [217]:
from display_entities import serve_ents
serve_ents(doc2 , matches, options = options)


    Serving on port 5000...
    Using the 'ent' visualizer



127.0.0.1 - - [01/Mar/2019 12:38:39] "GET / HTTP/1.1" 200 18275
127.0.0.1 - - [01/Mar/2019 12:38:39] "GET /favicon.ico HTTP/1.1" 200 18275



    Shutting down server on port 5000.



In [218]:
(matches)

[('JUR', 5, 6),
 ('JUR', 9, 10),
 ('JUR', 15, 16),
 ('JUR', 17, 18),
 ('JUR', 33, 36),
 ('JUR', 39, 40),
 ('JUR', 44, 45),
 ('JUR', 48, 51),
 ('JUR', 61, 62),
 ('JUR', 66, 67),
 ('JUR', 68, 70),
 ('JUR', 78, 79),
 ('JUR', 85, 88),
 ('JUR', 103, 105),
 ('JUR', 116, 119),
 ('JUR', 123, 126),
 ('JUR', 129, 130),
 ('JUR', 138, 139),
 ('JUR', 141, 142),
 ('JUR', 148, 149),
 ('JUR', 150, 151),
 ('JUR', 155, 159),
 ('JUR', 161, 162),
 ('JUR', 171, 172),
 ('JUR', 182, 184),
 ('JUR', 193, 194),
 ('JUR', 195, 197),
 ('JUR', 198, 200),
 ('JUR', 209, 211),
 ('JUR', 229, 231),
 ('JUR', 234, 235),
 ('JUR', 239, 241),
 ('JUR', 252, 255),
 ('JUR', 257, 258),
 ('JUR', 262, 263),
 ('JUR', 267, 269),
 ('JUR', 274, 275),
 ('JUR', 283, 285),
 ('JUR', 292, 293),
 ('JUR', 300, 303),
 ('JUR', 306, 307),
 ('JUR', 308, 309)]

In [224]:
entity_range = []
for label, start, end in matches:
    entity_range.append(set(range(start, end)))

In [225]:
entity_range

[{5},
 {9},
 {15},
 {17},
 {33, 34, 35},
 {35},
 {39},
 {44},
 {48, 49, 50},
 {50},
 {61},
 {66},
 {68},
 {68, 69},
 {78},
 {85, 86, 87},
 {87},
 {87, 88},
 {103, 104},
 {104},
 {116},
 {116, 117, 118},
 {123},
 {123, 124, 125},
 {129},
 {138},
 {141},
 {148},
 {150},
 {155},
 {155, 156, 157, 158},
 {155, 156, 157, 158},
 {161},
 {171},
 {182, 183},
 {183},
 {193},
 {195, 196},
 {196},
 {198, 199},
 {199},
 {209, 210},
 {210},
 {229},
 {229, 230},
 {234},
 {239, 240},
 {239, 240},
 {240},
 {252},
 {252, 253, 254},
 {257},
 {262},
 {267},
 {267, 268},
 {274},
 {283, 284},
 {292},
 {300},
 {300, 301, 302},
 {301, 302},
 {306},
 {308}]

In [226]:
        entity_range = [{61},
                        {66},
                        {68},
                        {68, 69},
                        {78},
                        {85, 86, 87},
                        {87},
                        {87, 88}]

In [233]:
check_overlap(6, entity_range)[0] in range(len(entity_range))

True

In [232]:
check_overlap(6, entity_range)[0] in range(entity_range)

5

In [234]:
matches

[('JUR', 5, 6),
 ('JUR', 9, 10),
 ('JUR', 15, 16),
 ('JUR', 17, 18),
 ('JUR', 33, 36),
 ('JUR', 35, 36),
 ('JUR', 39, 40),
 ('JUR', 44, 45),
 ('JUR', 48, 51),
 ('JUR', 50, 51),
 ('JUR', 61, 62),
 ('JUR', 66, 67),
 ('JUR', 68, 69),
 ('JUR', 68, 70),
 ('JUR', 78, 79),
 ('JUR', 85, 88),
 ('JUR', 87, 88),
 ('JUR', 87, 89),
 ('JUR', 103, 105),
 ('JUR', 104, 105),
 ('JUR', 116, 117),
 ('JUR', 116, 119),
 ('JUR', 123, 124),
 ('JUR', 123, 126),
 ('JUR', 129, 130),
 ('JUR', 138, 139),
 ('JUR', 141, 142),
 ('JUR', 148, 149),
 ('JUR', 150, 151),
 ('JUR', 155, 156),
 ('JUR', 155, 159),
 ('JUR', 155, 159),
 ('JUR', 161, 162),
 ('JUR', 171, 172),
 ('JUR', 182, 184),
 ('JUR', 183, 184),
 ('JUR', 193, 194),
 ('JUR', 195, 197),
 ('JUR', 196, 197),
 ('JUR', 198, 200),
 ('JUR', 199, 200),
 ('JUR', 209, 211),
 ('JUR', 210, 211),
 ('JUR', 229, 230),
 ('JUR', 229, 231),
 ('JUR', 234, 235),
 ('JUR', 239, 241),
 ('JUR', 239, 241),
 ('JUR', 240, 241),
 ('JUR', 252, 253),
 ('JUR', 252, 255),
 ('JUR', 257, 258),